In [1]:
import gym
import gym_coverage
import random
import numpy as np
import tensorflow as tf

from CNNSQLAgent import CNNSQLAgent
from collections import deque

from matplotlib import pyplot as plt

seed = 0
np.random.seed(seed)
random.seed(seed)

In [2]:
env = gym.make('Coverage-v1')
env.seed(seed)
N_F_pos = env.observation_space.spaces[0].n
N_F_map = env.observation_space.spaces[1].shape[0]
N_F = N_F_pos + N_F_map
N_A = env.action_space.n
agent = CNNSQLAgent(N_F_pos, N_F_map, N_A, epsilon_decay= 0.99999, memory_mode='NORMAL', batch_size=64)
#agent.restore_model(path="model/coverage_8_cnn_dqn_model.ckpt")

WARN: gym.spaces.Box autodetected dtype as <type 'numpy.float32'>. Please provide explicit dtype.


In [3]:
import time
max_t = 80
avg_return_list = deque(maxlen=10)
avg_loss_list = deque(maxlen=10)
for i in range(500000):
    obs = env.reset()
    obs = np.concatenate((np.eye(64)[obs[0]], obs[1].flatten()))
    done = False
    total_reward = 0
    total_loss = 0
    #env.render()
    #time.sleep(0.5)
    for t in range(max_t):
        action = agent.get_action(np.reshape(obs, (1, -1)))
        next_obs, reward, done, info = env.step(action)
        next_obs = np.concatenate((np.eye(64)[next_obs[0]], next_obs[1].flatten()))
        agent.add_experience(obs, action, reward, next_obs, done)
        
        loss = agent.train_model()
        agent.update_memory(t, max_t)
        agent.update_policy()
        #env.render()
        #time.sleep(0.5)
        obs = next_obs
        total_reward += reward
        total_loss += loss
        if done:
            break
            
    agent.update_target()
    avg_return_list.append(total_reward)
    avg_loss_list.append(total_loss)
    
    if (i%1000)==0:
        print('{} loss : {:.3f}, return : {:.3f}, eps : {:.3f}'.format(i, np.mean(avg_loss_list), np.mean(avg_return_list), agent.epsilon))
#agent.save_model(path="model/coverage_6_model_new.ckpt")
#agent.restore_model(path="model/coverage_6_model.ckpt")

0 loss : nan, return : -281.000, eps : 1.000
1000 loss : nan, return : -276.400, eps : 0.960


CNNSQLAgent.py:191: RuntimeWarning: divide by zero encountered in log
  next_v_value = self.alpha * np.log(np.sum(np.exp(next_q_value/ self.alpha), axis=1))
CNNSQLAgent.py:153: RuntimeWarning: invalid value encountered in less
  return np.random.choice(self.n_action, p=act_prob[0])


2000 loss : nan, return : -272.800, eps : 0.921
3000 loss : nan, return : -270.800, eps : 0.884
4000 loss : nan, return : -274.200, eps : 0.848
5000 loss : nan, return : -260.400, eps : 0.813
6000 loss : nan, return : -268.900, eps : 0.780
7000 loss : nan, return : -260.200, eps : 0.748
8000 loss : nan, return : -280.300, eps : 0.717
9000 loss : nan, return : -234.600, eps : 0.690
10000 loss : nan, return : -258.300, eps : 0.662
11000 loss : nan, return : -272.700, eps : 0.636
12000 loss : nan, return : -260.900, eps : 0.612
13000 loss : nan, return : -284.400, eps : 0.587
14000 loss : nan, return : -235.200, eps : 0.564
15000 loss : nan, return : -223.100, eps : 0.542
16000 loss : nan, return : -248.700, eps : 0.521
17000 loss : nan, return : -256.600, eps : 0.501
18000 loss : nan, return : -247.200, eps : 0.482
19000 loss : nan, return : -234.300, eps : 0.463
20000 loss : nan, return : -249.000, eps : 0.446


KeyboardInterrupt: 

In [4]:
agent.save_model(path="model/coverage_8_cnn_sql_model.ckpt")
max_t = 50
for i in range(100):
    obs = env.reset()
    obs = np.concatenate((np.eye(64)[obs[0]], obs[1].flatten()))
    done = False
    total_reward = 0
    env.render()
    time.sleep(0.25)
    for t in range(max_t):
        action = agent.get_action(np.reshape(obs, (1, -1)))
        next_obs, reward, done, info = env.step(action)
        
        next_obs = np.concatenate((np.eye(64)[next_obs[0]], next_obs[1].flatten()))
        
        obs = next_obs
        total_reward += reward
        env.render()
        time.sleep(0.25)
        if done:
            break
    print(total_reward)        

Model saved.
-261.0
-222.0
-300.0
-280.0
-262.0


KeyboardInterrupt: 